In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
warnings.simplefilter(action='ignore')

In [4]:
# Load the MovieLens 100K dataset
# Assuming the file is 'u.data' in the same directory
# Columns: user_id, movie_id, rating, timestamp
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Step 0: only select top 50 movies
valid_movies = np.array(data['movie_id'].value_counts().index[:50])
data = data[data['movie_id'].isin(valid_movies)]

# Step 1: Remove users s.t. count >= 2 * unique timestamp
temp = data.groupby('user_id')['timestamp'].agg(['count', 'nunique']).reset_index()
valid_users = temp[temp['count']/temp['nunique'] < 2]['user_id']
data = data[data['user_id'].isin(valid_users)]

# Step 2: Deduplicate and recode
data = data.groupby(['user_id', 'timestamp'], group_keys=False).apply(lambda group: group.sample(n=1, random_state=42))
data = data.sort_values(['user_id', 'timestamp']).reset_index(drop = True)
data['movie_id'] = pd.factorize(data['movie_id'])[0] + 1

num_users = data['user_id'].nunique()
num_movies = data['movie_id'].nunique()
num_ratings = len(data)

sparsity = 1 - (num_ratings / (num_users * num_movies))

print(f"Filtered Dataset: {num_users} users, {num_movies} movies")
print(f"Sparsity: {sparsity:.2%}")

Filtered Dataset: 902 users, 50 movies
Sparsity: 69.50%


In [5]:
data

,user_id,movie_id,rating,timestamp
0,1,1,5,874965478
1,1,2,5,874965706
2,1,3,5,874965739
3,1,4,5,874965758
4,1,5,5,874965954
...,...,...,...,...
13752,943,23,4,888639407
13753,943,13,5,888639427
13754,943,8,4,888692413
13755,943,41,4,888692699


In [6]:
np.random.seed(42)

all_users = data['user_id'].unique()
shuffled_indices = np.random.permutation(len(all_users))

midpoint = (3 * len(all_users)) // 4
indices_1 = shuffled_indices[:midpoint]
indices_2 = shuffled_indices[midpoint:]

train_users = shuffled_indices[indices_1]
test_users = shuffled_indices[indices_2]

In [7]:
data_train = data[data['user_id'].isin(train_users)].reset_index(drop = True)
data_test = data[data['user_id'].isin(test_users)].reset_index(drop = True)

In [8]:
# Prepare baskets
def prepare_baskets(data):
    return data.groupby("user_id")["movie_id"].apply(list).tolist()

train_baskets = prepare_baskets(data_train)
test_baskets = prepare_baskets(data_test)

train_baskets, val_baskets = train_test_split(train_baskets, test_size=0.25, random_state=42)

max_len = max(len(basket) for basket in train_baskets + val_baskets + test_baskets)

In [9]:
D = 32
batch_size = 256
max_epochs = 2000
lr = 1e-4
max_items = len(set(data_test['movie_id']))

In [10]:
len(set(data_test['movie_id']))

50

In [11]:
def preprocess_baskets(baskets):
    context_inputs = []
    target_inputs = []
    
    for basket in baskets:
        for idx, elt in enumerate(basket):            
            target_inputs.append(elt)
            context_inputs.append(basket[:idx] + [max_items + 1])
    
    context_inputs = pad_sequences(context_inputs, padding='post', maxlen = max_len, value=0)
    return np.array(context_inputs), np.array(target_inputs) - 1

train_context_input, train_target_input = preprocess_baskets(train_baskets)
val_context_input, val_target_input = preprocess_baskets(val_baskets)
test_context_input, test_target_input = preprocess_baskets(test_baskets)

In [12]:
input_context = layers.Input(shape=(max_len,), dtype=tf.int32, name="context_input")

alpha_embedding = layers.Embedding(input_dim=max_items + 2, output_dim=D, name="alpha_embedding")
context_embedding = alpha_embedding(input_context) 

class ZeroMaskEmbedding(layers.Layer):
    def call(self, embeddings, input_tokens):
        mask = tf.cast(tf.not_equal(input_tokens, 0), tf.float32) 
        mask = tf.expand_dims(mask, axis=-1) 
        return embeddings * mask 

context_embedding = ZeroMaskEmbedding()(context_embedding, input_context)

class SumLayer(layers.Layer):
    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=1) 

masked_embeddings = SumLayer()(context_embedding)

output = layers.Dense(max_items, activation="softmax", name="output_layer", use_bias = False)(masked_embeddings)

model = Model(inputs=input_context, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="sparse_categorical_crossentropy")

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_context_input, train_target_input,        
    validation_data=(val_context_input, val_target_input),
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping]
)

Epoch 1/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.9143 - val_loss: 3.9096
Epoch 2/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.9101 - val_loss: 3.9056
Epoch 3/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9056 - val_loss: 3.9018
Epoch 4/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.9001 - val_loss: 3.8981
Epoch 5/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8970 - val_loss: 3.8946
Epoch 6/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8922 - val_loss: 3.8911
Epoch 7/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8886 - val_loss: 3.8875
Epoch 8/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.8855 - val_loss: 3.8839
Epoch 9/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8809 - val_loss: 3.8801
Epoch 10/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.8758 - val_loss: 3.8763
Epoch 11/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8728 - val_loss: 3.8724
Epoch 12/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Epoch 69/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6559 - val_loss: 3.6792
Epoch 70/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6412 - val_loss: 3.6774
Epoch 71/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6405 - val_loss: 3.6755
Epoch 72/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6357 - val_loss: 3.6736
Epoch 73/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6296 - val_loss: 3.6717
Epoch 74/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6343 - val_loss: 3.6700
Epoch 75/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6360 - val_loss: 3.6682
Epoch 76/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6280 - val_loss: 3.6665
Epoch 77/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6207 - val_loss: 3.6648
Epoch 78/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6267 - val_loss: 3.6631
Epoch 79/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6280 - val_loss: 3.6615
Epoch 80/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5426 - val_loss: 3.5992
Epoch 137/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5273 - val_loss: 3.5984
Epoch 138/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5349 - val_loss: 3.5977
Epoch 139/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5223 - val_loss: 3.5970
Epoch 140/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5257 - val_loss: 3.5964
Epoch 141/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5206 - val_loss: 3.5957
Epoch 142/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5296 - val_loss: 3.5952
Epoch 143/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5263 - val_loss: 3.5944
Epoch 144/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5206 - val_loss: 3.5938
Epoch 145/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5211 - val_loss: 3.5930
Epoch 146/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5189 - val_loss: 3.5924
Epoch 147/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4640 - val_loss: 3.5652
Epoch 204/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4633 - val_loss: 3.5650
Epoch 205/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4687 - val_loss: 3.5646
Epoch 206/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4669 - val_loss: 3.5643
Epoch 207/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4568 - val_loss: 3.5639
Epoch 208/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4554 - val_loss: 3.5637
Epoch 209/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4621 - val_loss: 3.5632
Epoch 210/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4423 - val_loss: 3.5629
Epoch 211/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4653 - val_loss: 3.5627
Epoch 212/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4493 - val_loss: 3.5624
Epoch 213/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4585 - val_loss: 3.5621
Epoch 214/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4234 - val_loss: 3.5494
Epoch 271/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3991 - val_loss: 3.5494
Epoch 272/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4195 - val_loss: 3.5491
Epoch 273/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4021 - val_loss: 3.5491
Epoch 274/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4110 - val_loss: 3.5489
Epoch 275/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4278 - val_loss: 3.5487
Epoch 276/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4146 - val_loss: 3.5487
Epoch 277/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4140 - val_loss: 3.5485
Epoch 278/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4243 - val_loss: 3.5484
Epoch 279/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.4088 - val_loss: 3.5484
Epoch 280/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4148 - val_loss: 3.5482
Epoch 281/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3834 - val_loss: 3.5440
Epoch 338/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3797 - val_loss: 3.5440
Epoch 339/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3745 - val_loss: 3.5437
Epoch 340/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3783 - val_loss: 3.5436
Epoch 341/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3691 - val_loss: 3.5436
Epoch 342/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3927 - val_loss: 3.5435
Epoch 343/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3664 - val_loss: 3.5436
Epoch 344/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3834 - val_loss: 3.5435
Epoch 345/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3763 - val_loss: 3.5435
Epoch 346/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3833 - val_loss: 3.5434
Epoch 347/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.3758 - val_loss: 3.5433
Epoch 348/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context_input       │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha_embedding     │ (None, 44, 32)    │      1,664 │ context_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_mask_embedding │ (None, 44, 32)    │          0 │ alpha_embedding[… │
│ (ZeroMaskEmbedding) │                   │            │ context_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sum_layer           │ (None, 32)        │          0 │ zero_mask_embedd… │
│ (SumLayer)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 50)        │      1,600 │ sum_layer[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,794 (38.26 KB)

 Trainable params: 3,264 (12.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,530 (25.51 KB)

In [14]:
# Evaluate on Test Data
test_loss = model.evaluate(test_context_input, test_target_input, batch_size=batch_size)
print(f"Test Loss: {test_loss}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.5176 
Test Loss: 3.533583641052246
